In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


z = 5 # top-n rankings where z can be 1,3 or 5

dataset=pd.read_csv("/content/drive/MyDrive/Meta_Learning/meta_data.csv") #path for meta_dataset
dataset

In [ ]:
X=dataset.iloc[:,1:-2]
Y=dataset.iloc[:,-2:-1]
print(X)

In [ ]:
m = pd.read_csv("/content/drive/MyDrive/Meta_Learning/Models.csv") #name of the pre-trained models used
m

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=20) #dataset split for training the meta_learner model
X_train=np.array(X_train)
Y_train=np.array(Y_train)

UNCOMMENT FOR USING DECISION TREE

In [ ]:
reg=DecisionTreeRegressor(random_state=0) # training meta_learner
reg.fit(X_train, Y_train)

DecisionTreeRegressor(random_state=0)

UNCOMMENT FOR USING RANDOM FOREST

In [ ]:
# reg=RandomForestRegressor(n_estimators=50,random_state=0) # training meta_learner
# reg.fit(X_train, Y_train)

UNCOMMENT FOR USING SVR

In [ ]:
# reg=SVR(kernel = 'rbf') # training meta_learner
# reg.fit(X_train, Y_train)

In [ ]:
y_pred= reg.predict(X_test)
print(mean_squared_error(Y_test, y_pred))
print(r2_score(Y_test, y_pred))

For VEG 3

In [ ]:
data4=pd.read_csv("/content/drive/MyDrive/Meta_Learning/meta_Veg_3.csv") # path for meta_features extracted by probe network for plant disease classification task Veg3 
data4

In [ ]:
X4=data4.iloc[:,1:-2]
Y4=data4.iloc[:,-2:-1]

In [ ]:
y_4= reg.predict(X4)
veg_3 = pd.DataFrame(y_4, columns = ['predicted'])

In [ ]:
res_veg_3 = pd.concat([m,Y4,veg_3],axis=1) #dataframe with actual and predicted accuracies for different models
veg_3_pred = res_veg_3.drop(['Accuracy'], axis=1)
veg_3_actual = res_veg_3.drop(['predicted'], axis=1)

In [ ]:
veg_3_actual = veg_3_actual.sort_values(by=['Accuracy'], ascending=False) # sorting the actual rankings of the model
veg_3_pred = veg_3_pred.sort_values(by=['predicted'], ascending=False) # sorting the predicted rankings of the model

In [ ]:
veg_3_f1 = veg_3_actual["Models"]
veg_3_f1 = veg_3_f1[:z]
veg_3_f2 = veg_3_pred["Models"]
veg_3_f2 = veg_3_f2[:z]

In [ ]:
veg_3_f1 #actual rankings

In [ ]:
veg_3_f2 #predicted rankings 

COMPARING THE RANKINGS

In [ ]:
#Calculating the RBO score for predicted vs actual ranings

import math
def rbo(list1, list2, p=0.9):
   # tail recursive helper function
   def helper(ret, i, d):
       l1 = set(list1[:i]) if i < len(list1) else set(list1)
       l2 = set(list2[:i]) if i < len(list2) else set(list2)
       a_d = len(l1.intersection(l2))/i
       term = math.pow(p, i) * a_d
       if d == i:
           return ret + term
       return helper(ret + term, i + 1, d)
   k = max(len(list1), len(list2))
   x_k = len(set(list1).intersection(set(list2)))
   summation = helper(0, 1, k)
   return ((float(x_k)/k) * math.pow(p, k)) + ((1-p)/p * summation)

print("Veg3")
print(rbo(veg_3_f1,veg_3_f2))